In [2]:
import pandas as pd

FILE = "INFLUD23-17-06-2024.csv"

INFLUD = pd.read_csv(FILE, sep=';')
POPULATION = pd.read_csv("StatePopulation.csv")
POPULATION.drop(columns=["STATE"], inplace=True)

/tmp/ipykernel_53440/2690819796.py:5: DtypeWarning: Columns (15,18,20,23,53,61,62,63,72,74,79,91,93,109,130,143,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  INFLUD = pd.read_csv(FILE, sep=';')


In [8]:
import pickle
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, LabelEncoder

df = INFLUD.copy() 

# Filtra apenas os casos confirmados
df = df[df["CLASSI_FIN"].notna()]

# Agrupa por estado e data de notificação e conta o número de casos
df = df.groupby(["SG_UF_NOT", "DT_NOTIFIC"]).size().reset_index(name="CNT")

# Converte a data de notificação para o dia do ano
df["DOF_NOTIFIC"] = pd.to_datetime(df["DT_NOTIFIC"], format="%d/%m/%Y").dt.dayofyear
df.drop(columns="DT_NOTIFIC", inplace=True)


# Adiciona a população de cada estado
df = df.merge(POPULATION, left_on="SG_UF_NOT", right_on="UF")
df.drop(columns=["SG_UF_NOT"], inplace=True)

# Converte UF para numeral
uf_encoder = LabelEncoder()
df["UF"] = uf_encoder.fit_transform(df["UF"])
with open("encoders/uf.pickle", 'wb') as file:
    pickle.dump(uf_encoder, file, pickle.HIGHEST_PROTOCOL)

# risk_scaler = QuantileTransformer()
risk_scaler = MinMaxScaler()
df["RISK"] = df["CNT"] / df["POPULATION"]
df["RISK"] = risk_scaler.fit_transform(df[["RISK"]])

df.drop(columns=["CNT", "POPULATION"], inplace=True)

df

,DOF_NOTIFIC,UF,RISK
0,1,0,0.043576
1,32,0,0.109586
2,60,0,0.043576
3,91,0,0.109586
4,121,0,0.131589
...,...,...,...
9890,31,26,0.049226
9891,90,26,0.036812
9892,151,26,0.111298
9893,212,26,0.049226


In [4]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X = df[["DOF_NOTIFIC", "UF"]]
Y = df["RISK"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

linear_regression = LinearRegression()
linear_regression.fit(X_train, Y_train)

Y_pred = linear_regression.predict(X_test)

joblib.dump(linear_regression, "models/linear_regression.pkl")
mean_absolute_error(Y_test, Y_pred)

np.float64(0.05165227219032609)

In [21]:
date = pd.to_datetime("16/06/2024", format="%d/%m/%Y").dayofyear
uf = uf_encoder.transform(["SP"])[0]

y = linear_regression.predict([[date, uf]])
y

/home/kauan/Documents/Uni/machine-learning-abp/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([0.05457491])